In [137]:
import fastf1
import pandas as pd
from datetime import datetime

In [77]:
session=fastf1.get_session(2025, 12, 'Race')

logger      WARNING 	Failed to load schedule from FastF1 backend!
req            INFO 	No cached data found for season_schedule. Loading data...
_api           INFO 	Fetching season schedule...
req            INFO 	Data has been written to cache!


In [78]:
session

2025 Season Round 12: British Grand Prix - Race

In [79]:
session.load()

core           INFO 	Loading data for British Grand Prix - Race [v3.7.0]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No ca

In [81]:
session.results.columns

Index(['DriverNumber', 'BroadcastName', 'Abbreviation', 'DriverId', 'TeamName',
       'TeamColor', 'TeamId', 'FirstName', 'LastName', 'FullName',
       'HeadshotUrl', 'CountryCode', 'Position', 'ClassifiedPosition',
       'GridPosition', 'Q1', 'Q2', 'Q3', 'Time', 'Status', 'Points', 'Laps'],
      dtype='object')

In [82]:
df=session.laps

In [143]:
df[['Driver', 'DriverNumber', 'LapNumber', 'Stint', 'PitInTime', 'Position']].to_dict(orient='records')

[{'Driver': 'VER',
  'DriverNumber': '1',
  'LapNumber': 1.0,
  'Stint': 1.0,
  'PitInTime': NaT,
  'Position': 1.0},
 {'Driver': 'VER',
  'DriverNumber': '1',
  'LapNumber': 2.0,
  'Stint': 1.0,
  'PitInTime': NaT,
  'Position': 1.0},
 {'Driver': 'VER',
  'DriverNumber': '1',
  'LapNumber': 3.0,
  'Stint': 1.0,
  'PitInTime': NaT,
  'Position': 1.0},
 {'Driver': 'VER',
  'DriverNumber': '1',
  'LapNumber': 4.0,
  'Stint': 1.0,
  'PitInTime': NaT,
  'Position': 1.0},
 {'Driver': 'VER',
  'DriverNumber': '1',
  'LapNumber': 5.0,
  'Stint': 1.0,
  'PitInTime': NaT,
  'Position': 1.0},
 {'Driver': 'VER',
  'DriverNumber': '1',
  'LapNumber': 6.0,
  'Stint': 1.0,
  'PitInTime': NaT,
  'Position': 1.0},
 {'Driver': 'VER',
  'DriverNumber': '1',
  'LapNumber': 7.0,
  'Stint': 1.0,
  'PitInTime': NaT,
  'Position': 1.0},
 {'Driver': 'VER',
  'DriverNumber': '1',
  'LapNumber': 8.0,
  'Stint': 1.0,
  'PitInTime': NaT,
  'Position': 2.0},
 {'Driver': 'VER',
  'DriverNumber': '1',
  'LapNumber':

In [87]:
messages=session.race_control_messages

In [88]:
# penalty messages
messages[(messages.Message.str.contains('TIME PENALTY FOR')&(~messages.Message.str.contains('PENALTY SERVED')))]

,Time,Category,Message,Status,Flag,Scope,Sector,RacingNumber,Lap
115,2025-07-06 14:55:02,Other,FIA STEWARDS: 10 SECOND TIME PENALTY FOR CAR 8...,None,None,None,NaN,None,25
118,2025-07-06 14:59:11,Other,FIA STEWARDS: 10 SECOND TIME PENALTY FOR CAR 2...,None,None,None,NaN,None,27


In [ ]:
# events
messages[messages.Category=='SafetyCar'][['Time', 'Message', 'Status', 'Lap']]

,Time,Message,Status,Lap
67,2025-07-06 14:05:48,VIRTUAL SAFETY CAR DEPLOYED,DEPLOYED,2
72,2025-07-06 14:10:18,VIRTUAL SAFETY CAR ENDING,ENDING,4
81,2025-07-06 14:13:39,VIRTUAL SAFETY CAR DEPLOYED,DEPLOYED,5
87,2025-07-06 14:17:04,VIRTUAL SAFETY CAR ENDING,ENDING,7
97,2025-07-06 14:29:14,SAFETY CAR DEPLOYED,DEPLOYED,14
99,2025-07-06 14:38:36,SAFETY CAR IN THIS LAP,IN THIS LAP,17
102,2025-07-06 14:41:15,SAFETY CAR DEPLOYED,DEPLOYED,18
108,2025-07-06 14:49:02,SAFETY CAR IN THIS LAP,IN THIS LAP,21


In [105]:
# start and end time
first_lap=session.laps.iloc[0].LapStartDate
last_lap=session.laps.iloc[-1].LapStartDate

In [140]:
(first_lap-pd.Timedelta(hours=1))

Timestamp('2025-07-06 13:03:49.616000')

In [112]:
post_start_unix=int((first_lap-pd.Timedelta(hours=1)).timestamp())
post_end_unix=int((last_lap+pd.Timedelta(hours=1)).timestamp())

In [113]:
post_start_unix, post_end_unix

(1751807029, 1751820045)

## Try Reddit API

In [114]:
import requests

def fetch_new_posts(
    subreddit,
    limit=100,
    before=None,
    after=None,
    user_agent="my-reddit-script/1.0"
):
    base_url = f"https://www.reddit.com/r/{subreddit}/new.json"

    params = {
        "limit": limit
    }

    if before is not None:
        params["before"] = f"t3_{int(before)}"

    if after is not None:
        params["after"] = f"t3_{int(after)}"

    headers = {
        "User-Agent": user_agent
    }

    response = requests.get(base_url, params=params, headers=headers)
    response.raise_for_status()

    return response.json()


In [131]:
data = fetch_new_posts(
    subreddit="formula1",
    limit=1000,
    before=post_end_unix,
    after=post_start_unix
)


In [135]:
data['data']['children'][0]['data']['created_utc']

1767544434.0

In [136]:
data['data']['children'][-1]['data']['created_utc']

1767140069.0

In [138]:
datetime.utcfromtimestamp(1767544434.0)

<positron-console-cell-138>:1: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).


datetime.datetime(2026, 1, 4, 16, 33, 54)

In [1]:
# Imports
import requests
import pandas as pd

# ---------- CONFIG ----------
season = 2025
grand_prix = 12
session_type = "Race"  # Q=Qualifying, R=Race, FP1/FP2/FP3

# Map session_type to OpenF1 endpoints
session_map = {
    "Q": "qualifying",
    "R": "race",
    "FP1": "practice1",
    "FP2": "practice2",
    "FP3": "practice3"
}
session_endpoint = session_map.get(session_type, "race")

# ---------- FETCH DATA ----------
url = f"https://api.openf1.io/v1/sessions/{season}/{grand_prix}/{session_endpoint}/laps"
response = requests.get(url)
response.raise_for_status()  # raise error if request fails

laps_data = response.json()  # list of lap dictionaries

# Check if data exists
if not laps_data:
    print("No lap data returned from OpenF1")
else:
    print(f"Retrieved {len(laps_data)} laps")

# ---------- PROCESS DATA ----------
laps_df = pd.DataFrame(laps_data)

# Keep only relevant columns
columns = ['Driver', 'DriverNumber', 'LapNumber', 'Stint', 'PitInTime', 'Position']
laps_df = laps_df[[c for c in columns if c in laps_df.columns]]

# Convert PitInTime to ISO format
if 'PitInTime' in laps_df.columns:
    laps_df['PitInTime'] = pd.to_datetime(laps_df['PitInTime'], errors='coerce').apply(
        lambda x: x.isoformat() if pd.notnull(x) else None
    )

# Convert numeric types to native Python types
laps_df = laps_df.applymap(lambda x: x.item() if hasattr(x, "item") else x)

# ---------- OUTPUT ----------
# Display first 5 rows in Jupyter
laps_df.head()


ConnectionError: HTTPSConnectionPool(host='api.openf1.io', port=443): Max retries exceeded with url: /v1/sessions/2025/12/race/laps (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x137c63380>: Failed to resolve 'api.openf1.io' ([Errno 8] nodename nor servname provided, or not known)"))

In [3]:
import requests
try:
    r = requests.get("https://api.openf1.org/v1/sessions/2025/12/race/laps")
    print(r.status_code)
except Exception as e:
    print(e)


400


## Try Reddit API v2

In [26]:
import requests
try:
    r = requests.get('https://www.reddit.com/r/formula1/comments/1lt1q7f/2025_british_gp_race_discussion/.json')
    print(r.status_code)
except Exception as e:
    print(e)

200


In [27]:
data=r.json()

In [28]:
len(data[1]['data']['children'])

84